In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('data/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv')


/var/folders/3k/phn3qb093cj5b3twlzg9smrh0000gq/T/ipykernel_24107/1523784344.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv')


In [16]:
#Step 2: Data Exploration
#The below will:

#Tell us the names of the columns in our data.
#The datatypes in said columns.
#Check for missing values.
#Check for duplicate rows.
#Provide some statistical summary in our data.
#Count the number of occurunces of unique values in our data's columns.


print("Columns:", '\n',df.columns)
print("Data Types:", '\n',df.dtypes)
print("Null Counts:",'\n',df.isnull().sum())
print("Duplicate Columns:", '\n',df.duplicated().sum())
print("Some Statistical Numbers:", '\n',df.describe())
#df['Prscrbr_State_Abrvtn'].value_counts()

Columns: 
 Index(['Prscrbr_NPI', 'Prscrbr_Last_Org_Name', 'Prscrbr_First_Name',
       'Prscrbr_City', 'Prscrbr_State_Abrvtn', 'Prscrbr_State_FIPS',
       'Prscrbr_Type', 'Prscrbr_Type_Src', 'Brnd_Name', 'Gnrc_Name',
       'Tot_Clms', 'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst',
       'Tot_Benes', 'GE65_Sprsn_Flag', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills',
       'GE65_Tot_Drug_Cst', 'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag',
       'GE65_Tot_Benes'],
      dtype='object')
Data Types: 
 Prscrbr_NPI                int64
Prscrbr_Last_Org_Name     object
Prscrbr_First_Name        object
Prscrbr_City              object
Prscrbr_State_Abrvtn      object
Prscrbr_State_FIPS        object
Prscrbr_Type              object
Prscrbr_Type_Src          object
Brnd_Name                 object
Gnrc_Name                 object
Tot_Clms                   int64
Tot_30day_Fills          float64
Tot_Day_Suply              int64
Tot_Drug_Cst             float64
Tot_Benes                float64


In [17]:
#Data Wrangling (required)


# Identify rows that describe claims for buprenorphine
buprenorphine_claims = df[df['Brnd_Name'].str.contains('buprenorphine', na=False, case=False) | df['Gnrc_Name'].str.contains('buprenorphine', na=False, case=False)]

# Aggregate per state usage of buprenorphine
buprenorphine_usage_by_state = buprenorphine_claims.groupby('Prscrbr_State_Abrvtn')['Tot_Day_Suply'].sum()

# Top 5 states by volume of buprenorphine usage
top_states = buprenorphine_usage_by_state.sort_values(ascending=False).head(5)
print(top_states)


Prscrbr_State_Abrvtn
CA    3244694
PA    2880656
FL    2671723
MA    2447394
NC    2382440
Name: Tot_Day_Suply, dtype: int64


In [30]:
#Data Visualization (optional)


import os
import plotly.graph_objects as go
import plotly.io as pio


#Read data
df = pd.read_csv('data/MUP_DPR_RY23_P04_V10_DY21_NPIBN.csv')


#Creating claims dataframe
buprenorphine_claims = df[df['Brnd_Name'].str.contains('buprenorphine', na=False, case=False) | df['Gnrc_Name'].str.contains('buprenorphine', na=False, case=False)]


# Sum total days supply by state
state_usage = buprenorphine_claims.groupby('Prscrbr_State_Abrvtn')['Tot_Day_Suply'].sum().reset_index()
state_usage.columns = ['State', 'Total Usage']

# Sort by Total Usage
state_usage = state_usage.sort_values('Total Usage', ascending=False)

# Create a new column to determine color based on whether the state is in the top 5
state_usage['color'] = ['Top 5' if i < 5 else 'Other' for i in range(state_usage.shape[0])]

# Create the bar chart
fig = go.Figure()

for color in state_usage['color'].unique():
    fig.add_trace(go.Bar(
        x=state_usage.loc[state_usage['color'] == color, 'State'], 
        y=state_usage.loc[state_usage['color'] == color, 'Total Usage'],
        name=color,
        marker_color='red' if color=='Top 5' else 'blue'
    ))

fig.update_layout(title='Buprenorphine Usage by State', 
                  xaxis_title='',
                  yaxis_title='Total Usage (days supply)',
                  )

# Show the figure
fig.show()

# Save the figure as a jpg
pio.write_image(fig, 'buprenorphine_usage_viz.jpg')
